In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt

epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("Epochs:", epochs)

In [ ]:
# -------------------------------
# 1. 전처리
# -------------------------------
root_dir = os.path.expanduser("./archive/chest_xray")
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # 흑백(1채널)로 변환
    transforms.Resize((128, 128)),                # 128 by 128 크기로 변환
    transforms.ToTensor()                         # 이미지를 PyTorch Tensor로 변환
])
input_dim = 128 * 128  # 모델의 입력 차원

# train/val/test dataset 구성
trainval_ds = datasets.ImageFolder(os.path.join(root_dir, 'train'), transform=transform)  # train+val
test_ds = datasets.ImageFolder(os.path.join(root_dir, 'test'),  transform=transform)      # test

# train/val 분할
train_size = int(0.8 * len(trainval_ds))  # train set의 사이즈
val_size = len(trainval_ds) - train_size  # val set의 사이즈
train_ds, val_ds = random_split(trainval_ds, [train_size, val_size], generator=torch.Generator().manual_seed(42))
print("train:", train_size, "val:", val_size, "test:", len(test_ds))

# DataLoader 구성 (batch_size=32, iteration마다 32개 이미지 처리)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)  # shuffle: 과적합 방지 및 일반화 성능 향상 
val_loader = DataLoader(val_ds, batch_size=32)  
test_loader = DataLoader(test_ds, batch_size=32)

In [ ]:
# -------------------------------
# 2-1. Logistic 모델 정의
# -------------------------------
class LogisticModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)  # binary classficaiton

    def forward(self, x):
        return torch.sigmoid(self.linear(x))  # sigmoid 함수로 확률 출력 (0 ~ 1)

In [ ]:
# -------------------------------
# 2-2. DeepNN 모델 정의
# -------------------------------
class DeepNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# -------------------------------
# 3-1. 학습 진행하는 함수 (학습)
# -------------------------------
def train(model, loader, optimizer, criterion):
    model.train()  # 학습 모드 활성화
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in loader:
        images = images.view(images.size(0), -1).to(device)  # Flatten
        labels = labels.float().unsqueeze(1).to(device)  # [32] → [32,1]

        optimizer.zero_grad()              # grad 초기화
        outputs = model(images)            # forward
        loss = criterion(outputs, labels)  # loss 계산
        loss.backward()                    # backward
        optimizer.step()                   # update model param

        running_loss += loss.item() * labels.size(0)
        preds = (outputs >= 0.5).float()   # sigmod → 0 or 1
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
# -------------------------------
# 3-2. 학습 진행하는 함수 (검증)
# -------------------------------
def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.view(images.size(0), -1).to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)

            preds = (outputs >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
# -------------------------------
# 4. 테스트 진행 함수
# -------------------------------
def evaluate(model, loader):
    model.eval()  # 평가 모드
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.view(images.size(0), -1).to(device)  # Flatten
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)

            preds = (outputs >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy

In [ ]:
# -------------------------------
# 5. 학습 함수를 통해 학습 
# -------------------------------

# 저장용 리스트
lr_train_accs, lr_val_accs = [], []
lr_train_losses, lr_val_losses = [], []

dnn_train_accs, dnn_val_accs = [], []
dnn_train_losses, dnn_val_losses = [], []

In [ ]:
# LR
model1 = LogisticModel(input_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

for epoch in range(epochs):
    train_loss, train_acc = train(model1, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model1, val_loader, criterion)

    lr_train_losses.append(train_loss)
    lr_val_losses.append(val_loss)
    lr_train_accs.append(train_acc)
    lr_val_accs.append(val_acc)

    print(f"[LR][Epoch {epoch+1}] Train Acc: {train_acc:.2f}, Train Loss: {train_loss:.2f}")
    print(f"[LR][Epoch {epoch+1}] Val Acc: {val_acc:.2f}, Val Loss: {val_loss:.2f}")

In [ ]:
# DNN
model2 = DeepNN(input_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001, weight_decay=1e-4)

for epoch in range(epochs):
    train_loss, train_acc = train(model2, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model2, val_loader, criterion)

    dnn_train_losses.append(train_loss)
    dnn_val_losses.append(val_loss)
    dnn_train_accs.append(train_acc)
    dnn_val_accs.append(val_acc)

    print(f"[DNN][Epoch {epoch+1}] Train Acc: {train_acc:.2f}, Train Loss: {train_loss:.2f}")
    print(f"[DNN][Epoch {epoch+1}] Val Acc: {val_acc:.2f}, Val Loss: {val_loss:.2f}")

In [ ]:
# -------------------------------
# 6. 테스트를 통해 정확도 도출
# -------------------------------

# LR
lr_test_acc = evaluate(model1, test_loader)
print(f"[LR] Test Acc: {lr_test_acc:.2f}")

# DNN
dnn_test_acc = evaluate(model2, test_loader)
print(f"[DNN] Test Acc: {dnn_test_acc:.2f}")


In [ ]:
# -------------------------------
# 7-1. 시각화 (Flow Graph)
# -------------------------------

epochs_range = range(1, epochs + 1)
plt.figure(figsize=(12, 8))

# 1. Train Loss: LR vs DNN
plt.subplot(2, 2, 1)
plt.plot(epochs_range, lr_train_losses, label='LR', linestyle='-')
plt.plot(epochs_range, dnn_train_losses, label='DNN', linestyle='-')
plt.title('Train Loss (LR vs DNN)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# 2. Val Loss: LR vs DNN
plt.subplot(2, 2, 2)
plt.plot(epochs_range, lr_val_losses, label='LR', linestyle='-')
plt.plot(epochs_range, dnn_val_losses, label='DNN', linestyle='-')
plt.title('Val Loss (LR vs DNN)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# 3. Train Accuracy: LR vs DNN
plt.subplot(2, 2, 3)
plt.plot(epochs_range, lr_train_accs, label='LR', linestyle='-')
plt.plot(epochs_range, dnn_train_accs, label='DNN', linestyle='-')
plt.title('Train Accuracy (LR vs DNN)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# 4. Val Accuracy: LR vs DNN
plt.subplot(2, 2, 4)
plt.plot(epochs_range, lr_val_accs, label='LR', linestyle='-')
plt.plot(epochs_range, dnn_val_accs, label='DNN', linestyle='-')
plt.title('Val Accuracy (LR vs DNN)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# -------------------------------
# 7-2. 시각화 (Bar Plot)
# -------------------------------

model_names = ["LR", "DNN"]
train_accs = [lr_train_accs[-1], dnn_train_accs[-1]]
val_accs = [lr_val_accs[-1], dnn_val_accs[-1]]
test_accs = [lr_test_acc, dnn_test_acc]

x = np.arange(len(model_names))
bar_width = 0.25

plt.figure(figsize=(6, 4))
plt.bar(x - bar_width, train_accs, width=bar_width, label="Train")
plt.bar(x, val_accs, width=bar_width, label="Val")
plt.bar(x + bar_width, test_accs, width=bar_width, label="Test")

plt.xticks(x, model_names)
plt.ylabel("Accuracy")
plt.ylim(0.5, 1.0)
plt.title("Chest X-ray Binary Classfication")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()